# Cancer patient readmission model training
- Data:  \
18 months cancer patient EHR\
Training set / Test set : 0.8/0.2
- Model:\
Baseline：Logistic regression, class_weight="balanced",solver="liblinear", penalty="l2"\
Tree-based：XGBoost scale_pos_weight = neg/pos，max depth = 4,  XGBoost Temp scaling
- Features:\
Demographic：年龄、性别、医保\
Basic info：Admission type, Length of stay, 出院 disposition\
Cancer type:（ICD10 大类分组）\
Surgery:（有无/等级/出院间隔）\
Treatment：化疗方案、放疗、免疫/靶向\
出院前 48–72h 动态信号：抗菌药升级、G-CSF、氧疗、输血、白蛋白、病程关键词（发热、感染、腹水、黄疸、出血等）\
病理/分子病理特征:（EGFR/ALK/PD-L1 等，转为二元变量）

## Import the libraries

In [1]:
import pandas as pd
import numpy as np
import yaml
import os
import data_cleaning_func as dcf
import re

## Identify readmission patients

In [2]:
CFG = yaml.safe_load(open('/Users/lajikf/Desktop/Python/Projectone/configs/excel_ingest.yaml','r',encoding='utf-8'))
def get_readmission_patients(df, cancer_codes, days_threshold=30, cfg = CFG, emergency = 'all'):
    """
    Identify patients who were readmitted within time limits after chemotherapy.
    This function does NOT filter for chemotherapy procedure. Use orders to filter chemotherapy visit_sn.
    Args:
        df (pd.DataFrame): DataFrame containing patient admission data.
        cancer_codes (list): ICD-10 code of cancer included.
        days_threshold (int): Number of days to consider for readmission.
        emergency (str): Code indicating emergency readmission. all for all, Y for emergency only, N for non-emergency only.
    Returns:
        (readmit_visit_sn,first_admit_visit_sn) (tuple): Tuple of two lists. 
        readmit_patient is visit_sn readmitted within the threshold after chemotherapy.
        first_admit_visit_sn is visit_sn of the first admission.
    """
    ## Convert date columns to datetime
    print(f'Getting readmission patients with cancer-code{cancer_codes}')
    df['B12'] = pd.to_datetime(df['B12'], errors='coerce')
    df['B15'] = pd.to_datetime(df['B15'], errors='coerce')
    if df['B12'].isna().sum() > 0:
        print("Date type conversion error in B12:")
        print(df[df['B12'].isna()][['patient_id','visit_sn','B12']])
    else:
        if df['B15'].isna().sum() > 0:
            print("date type conversion error in B15:")
            print(df[df['B15'].isna()][['patient_id','visit_sn','B15']])
        else:
            print("Date type conversion successful!")
    # df = df[['visit_sn','patient_id','B12','B15','B11C','C03C']].copy() # check the list, which colmn to keep

    ## calculate next visit date to identify readmission
    df = df.sort_values(by=['patient_id','B12'])
    df['next_admit_date'] = df.groupby('patient_id')['B12'].shift(-1)
    # df['next_admit_type'] = df.groupby('patient_id')['B11C'].shift(-1)
    df['next_visit_sn'] = df.groupby('patient_id')['visit_sn'].shift(-1)
    df = df[df['next_admit_date'].notna()]
    df['days_to_next_admit'] = df['next_admit_date'] - df['B15']
    df = df[df['days_to_next_admit'].dt.days < days_threshold]
    # df = df[df['C03C'].str.contains('|'.join(cancer_codes))] # filter cancer diagnosis
    if emergency == 'all':
        print(f'Total readmission patients in {days_threshold} days with cancer diag: {df["patient_id"].nunique()}\nTotal readmission visits in {days_threshold} days with cancer diag: {df["visit_sn"].nunique()}\n')
    elif emergency == 'Y':
        df = df[df['next_admit_type'] == '1']
        print(f'Total readmission EMERGENCY patients in {days_threshold} days with cancer diag: {df["patient_id"].nunique()}\nTotal readmission EMERGENCY visit in {days_threshold} days with cancer diag: {df["visit_sn"].nunique()}\n')
    elif emergency == 'N':
        df = df[df['next_admit_type'] != '1']
        print(f'Total readmission NON-EMERGENCY patients in {days_threshold} days with cancer diag: {df["patient_id"].nunique()}\nTotal readmission NON-EMERGENCY visits in {days_threshold} days with cancer diag: {df["visit_sn"].nunique()}\n')
    readmit_visit_sn_list = df['next_visit_sn'].unique().tolist()
    first_admit_patient_list = df['visit_sn'].unique().tolist()
    return list(zip(readmit_visit_sn_list, first_admit_patient_list)), readmit_visit_sn_list, first_admit_patient_list
    

## Get the patients in the first 4 months to check readmission rate and choosing positive samples

In [ ]:
facesheet_202401 = pd.read_excel('.../facesheet.xlsx')
facesheet_202402 = pd.read_excel('.../facesheet.xlsx')
facesheet_202403 = pd.read_excel('.../facesheet.xlsx')
facesheet_202404 = pd.read_excel('.../facesheet.xlsx')
facesheet_202401to202404 = pd.concat([facesheet_202401,facesheet_202402,facesheet_202403,facesheet_202404])

In [ ]:
lung_cancer_code = []
tuple_30,list_30,list_30_firsttime = get_readmission_patients(facesheet_202401to202404,lung_cancer_code)
tuple_15,list_15,list_15_firsttime = get_readmission_patients(facesheet_202401to202404,lung_cancer_code, days_threshold=15)
tuple_7,list_7,list_7_firsttime = get_readmission_patients(facesheet_202401to202404,lung_cancer_code, days_threshold=7)
list_emer = get_readmission_patients(facesheet_202401to202404,lung_cancer_code,emergency='Y')
print(list_30)

In [ ]:
## Save the readmit patient list in order to check their diag
df15 = pd.DataFrame(list_15, columns=['visit_sn'])
cancer15 = facesheet_202401to202404[facesheet_202401to202404.visit_sn.isin(df15.visit_sn)]
cancer15.to_excel('/Users/lajikf/Desktop/Python/Projectone/data/cancer15.xlsx', index=False)
df30 = pd.DataFrame(list_30, columns=['visit_sn'])
cancer30 = facesheet_202401to202404[facesheet_202401to202404.visit_sn.isin(df30.visit_sn)]
cancer30.to_excel('/Users/lajikf/Desktop/Python/Projectone/data/cancer30.xlsx', index=False)

### Get the chemo patient list and filter the readmit patient list

In [ ]:
drg = pd.read_excel('.../2024DRG.xlsx')
temp_drg =pd.read_excel('.../2025DRG.xlsx')
drg = pd.concat([drg,temp_drg])
drg['WR'] = pd.to_numeric(drg['WR'], errors='coerce')  
drg = drg.sort_values(by ='WR', ascending=False)


In [ ]:
info_202401 = pd.read_excel('.../basic_info.xlsx')
info_202402 = pd.read_excel('.../basic_info.xlsx')
info_202403 = pd.read_excel('.../basic_info.xlsx')
info_202404 = pd.read_excel('.../basic_info.xlsx')
info_202401to202404 = pd.concat([info_202401,info_202402,info_202403,info_202404])

In [ ]:
chemo_drg = drg[drg['all_surgery_concat'].str.contains('化疗')]
chemo_202401to202404 = info_202401to202404[info_202401to202404.medical_record_no.isin(chemo_drg.medical_record_no)]
chemo_visit_sn_202401to202404 = chemo_202401to202404['visit_sn']

list_chemo_30_firsttime = pd.Series(list_30_firsttime)[pd.Series(list_30_firsttime).isin(chemo_visit_sn_202401to202404)]
list_chemo_30_firsttime = [str(x).split('.')[0] for x in list_chemo_30_firsttime.tolist()]
list_chemo_30_firsttime = [int(x) for x in list_chemo_30_firsttime]
print("list_chemo_30_firsttime =", len(list_chemo_30_firsttime))
print(list_chemo_30_firsttime[:5])

first_readmit_dict = {k:v for v,k in tuple_30} # dict: first admit visit_sn as key, readmit visit_sn as value
list_chemo_30_readmit = [first_readmit_dict[x] for x in list_chemo_30_firsttime]
list_chemo_30_readmit = [str(x).split('.')[0] for x in list_chemo_30_readmit]
list_chemo_30_readmit = [int(x) for x in list_chemo_30_readmit]
print("list_chemo_30_readmit =", len(list_chemo_30_readmit))
print(list_chemo_30_readmit[:5])

facesheet_chemo_30_readmit = facesheet_202401to202404[facesheet_202401to202404['visit_sn'].isin(list_chemo_30_readmit)]
print('cancer 30 with chemo shape =', facesheet_chemo_30_readmit.shape)
# master_chemo_30_readmit.to_excel('/Users/lajikf/Desktop/Python/Projectone/data/master_chemo_30_readmit.xlsx', index=False)



### Find the patient without chemotherapy

In [ ]:
no_chemo_drg = drg[~drg['all_surgery_concat'].str.contains('化疗')]
no_chemo_202401to202404 = info_202401to202404[info_202401to202404['medical_record_no'].isin(no_chemo_drg['medical_record_no'])]
print('no chemo patient shape =', no_chemo_202401to202404.shape)
list_chemo_30_readmitwithoutchemo = [x for x in list_chemo_30_readmit if x not in no_chemo_202401to202404['visit_sn'].values]
print('list_chemo_30_readmitwithoutchemo =', len(list_chemo_30_readmitwithoutchemo))
master_chemo_30_readmitwithoutchemo = facesheet_202401to202404[facesheet_202401to202404['visit_sn'].isin(list_chemo_30_readmitwithoutchemo)]
master_chemo_30_readmitwithoutchemo.to_excel('/Users/lajikf/Desktop/Python/Projectone/data/master_chemo_30_readmitwithoutchemo.xlsx', index=False)

### Find the patient with emergency readmission

In [ ]:
emer_drg = drg[drg['inpatient_type']=='急诊']
emer_drg = pd.merge(emer_drg, info_202401to202404[['patient_id','medical_record_no','visit_sn']], on='medical_record_no', how='inner')
print(emer_drg.head(5))
print(emer_drg.shape)
emer_drg_readmit = pd.merge(emer_drg, pd.DataFrame(list_30, columns=['visit_sn']), on='visit_sn', how='inner')
print('emer_drg_readmit shape =', emer_drg_readmit.shape)
print(emer_drg_readmit)

### Find the readmission with chemo

In [ ]:
list_chemo_15_firsttime = pd.Series(list_15_firsttime)[pd.Series(list_15_firsttime).isin(chemo_visit_sn_202401to202404)]
list_chemo_15_firsttime = [str(x).split('.')[0] for x in list_chemo_15_firsttime.tolist()]
list_chemo_15_firsttime = [int(x) for x in list_chemo_15_firsttime]
print("list_chemo_15_firsttime =", len(list_chemo_15_firsttime))
print(list_chemo_15_firsttime[:5])

first_readmit_dict = {k:v for v,k in tuple_30} 
list_chemo_15_readmit = [first_readmit_dict[x] for x in list_chemo_15_firsttime]
list_chemo_15_readmit = [str(x).split('.')[0] for x in list_chemo_15_readmit]
list_chemo_15_readmit = [int(x) for x in list_chemo_15_readmit]
print("list_chemo_15_readmit =", len(list_chemo_15_readmit))
print(list_chemo_15_readmit[:5])

master_chemo_15_readmit = facesheet_202401to202404[facesheet_202401to202404['visit_sn'].isin(list_chemo_15_readmit)]
print('cancer 15 with chemo shape =', master_chemo_15_readmit.shape)
master_chemo_15_readmit.to_excel('/Users/lajikf/Desktop/Python/Projectone/data/master_chemo_15_readmit.xlsx', index=False)

In [ ]:
drg_202401to202404 = pd.merge(drg, info_202401to202404[['patient_id','medical_record_no','visit_sn']], on='medical_record_no', how='inner')
drg_202401to202404.head(5)
drg_tuple_readmit_30,drg_list_readmit_30,drg_list_readmit_30_firsttime = get_readmission_patients(drg_202401to202404,lung_cancer_code,days_threshold=30)

emer_drg = drg[drg['inpatient_type']=='急诊']
emer_drg = pd.merge(emer_drg, info_202401to202404[['patient_id','medical_record_no','visit_sn']], on='medical_record_no', how='inner')
unplan_drg = drg[drg['unplaned'] == '是']
unplan_drg = pd.merge(unplan_drg, info_202401to202404[['patient_id','medical_record_no','visit_sn']], on='medical_record_no', how='inner')
emer_and_unplan_drg = pd.concat([emer_drg, unplan_drg]).drop_duplicates()

temp_emer_drg_readmit = pd.merge(emer_drg, pd.DataFrame(drg_list_readmit_30, columns=['visit_sn']), on='visit_sn', how='inner')
print('emer readmit,', temp_emer_drg_readmit.shape)

emer_and_unplan_drg_readmit = pd.merge(emer_and_unplan_drg, pd.DataFrame(drg_list_readmit_30, columns=['visit_sn']), on='visit_sn', how='inner')
print('emer + unplanned readmit,', emer_and_unplan_drg_readmit.shape)


### Summary in choosing positive samples
- There is too little sample when only using the emergency readmission (21 readmission in 4 months). 
- When using readmission with chemo therapy, sample size is enough (241 readmission in 4 months) but cannot distinguish planned chemo therapy which sometimes have a 7 days or 15 days interval depending on the plan. 
- Combining emergency readmission with unplanned inpatient (categorized in DRG system), sample size is moderate (112 readmission in 4 months) while ensuring the qualiity of those positive samples. \
Thus choose emergency + unplanned as positive sample


## Construct dataframe for analysis 

### Read the files

In [ ]:
## Args needed for file construction
MON = 18
DISKDIR = '/Users/lajikf/Desktop/Python/Projectone/data/raw'
FLASHDRIVE = '/Volumes/FENG/Projectone'
CACHEDIR = '/Users/lajikf/Desktop/Python/Projectone/data/cache'

TIMEFRAME = []
for year in [2024,2025]:
    for month in range(12):
        if month < 9:
            TIMEFRAME.append(str(year)+'0'+str(month+1))
        else:
            TIMEFRAME.append(str(year)+str(month+1))
TIMEFRAME = TIMEFRAME[:MON]


In [ ]:
## Construct facesheet dataframe
facesheet = dcf.read_concat(TIMEFRAME,DISKDIR,'face_sheet.xlsx',None)
facesheet.reset_index(drop=True, inplace=True)
facesheet.to_feather(CACHEDIR + '/facesheet.feather')

In [ ]:
## Construct the info dataframe
info = dcf.read_concat(TIMEFRAME,FLASHDRIVE,'info.xlsx',None)
info.reset_index(drop=True, inplace=True)
info.to_feather(CACHEDIR + '/info.feather')

In [ ]:
## Construct the DRG dataframe
drg = pd.read_excel('/Users/lajikf/Desktop/Python/Projectone/data/raw/2024DRG.xlsx')
temp_drg =pd.read_excel('/Users/lajikf/Desktop/Python/Projectone/data/raw/2025DRG.xlsx')
drg = pd.concat([drg,temp_drg])
del temp_drg
drg['WR'] = pd.to_numeric(drg['WR'], errors='coerce')  # 把无法转为数字的值变为 NaN
drg = drg.sort_values(by ='WR', ascending=False)
drg.reset_index(drop=True, inplace=True)
drg.drop_duplicates(subset = ['medical_record_no'], keep='first', inplace=True)
drg = pd.merge(info[['patient_id','medical_record_no','visit_sn']],drg , on='medical_record_no', how='inner')
drg[['B12']] = drg[['B12']].apply(pd.to_datetime, errors='coerce')
drg[['B15']] = drg[['B15']].apply(pd.to_datetime, errors='coerce')
drg[['surgery_date']] = drg[['surgery_date']].apply(pd.to_datetime, errors='coerce')
drg['inhospital_days'] = (drg['B15'] - drg['B12']).dt.days
drg.to_feather(CACHEDIR + '/drg.feather')

## emer_and_unplan_and_transaction_drg
emer_drg = drg[drg['inpatient_type']=='急诊']
unplan_drg = drg[drg['unplaned'] == '是']
transaction_drg = drg[drg['inpatient_type']=='其他医疗机构转入']
emer_and_unplan_drg = pd.concat([emer_drg, unplan_drg]).drop_duplicates()
emer_and_unplan_trans_drg = pd.concat([emer_drg,unplan_drg,transaction_drg]).drop_duplicates()

In [ ]:
## Construct the medical note dataframe
note  = dcf.read_concat(TIMEFRAME,FLASHDRIVE,'note.xlsx',features=None)
note.reset_index(drop=True, inplace=True)
note['record_datetime'] = note['record_datetime'].apply(pd.to_datetime, errors='coerce')
note.to_feather(CACHEDIR + '/note.feather')

### Get the readmit patient

In [ ]:
cancer_code = ['C11','C16','C18','C20','C22','C23','C24','C25','C34','C50','C56','C61','C64','C67','C71','C79','C82','C83','C85','C90','C92','D46']
tuple_master,readmit_18m,readmit_18m_firsttime = dcf.get_readmission_patients(facesheet,cancer_code, days_threshold=30)
emer_and_unplan_drg_readmit_18m = pd.merge(emer_and_unplan_drg, pd.DataFrame(readmit_18m, columns=['visit_sn']), on='visit_sn', how='inner')
emer_and_unplan_trans_drg_readmit_18m = pd.merge(emer_and_unplan_trans_drg,pd.DataFrame(readmit_18m, columns=['visit_sn']), on='visit_sn', how='inner')

### Construct analysis dataframe with basic info, DRG, and face sheet

In [ ]:
analysis_df = pd.DataFrame(facesheet['visit_sn'])
analysis_df = analysis_df.drop_duplicates()
analysis_df = analysis_df.reset_index(drop=True)
analysis_df = pd.merge(analysis_df, info[['patient_id','visit_sn','medical_record_no']], on = 'visit_sn' , how = 'inner')
analysis_df['readmit_30d'] = analysis_df['visit_sn'].apply(lambda x: 1 if x in readmit_18m else 0)
analysis_df['readmit_30d_emer_unplan'] = analysis_df['visit_sn'].apply(lambda x: 1 if x in emer_and_unplan_drg_readmit_18m['visit_sn'].values else 0)
analysis_df['readmit_30d_emer_unplan_trans'] = analysis_df['visit_sn'].apply(lambda x: 1 if x in emer_and_unplan_trans_drg_readmit_18m['visit_sn'].values else 0)
analysis_df = pd.merge(analysis_df, drg, on='medical_record_no', how='left', suffixes=('', '_drop'))
analysis_df = analysis_df.loc[:, ~analysis_df.columns.str.endswith('_drop')]
analysis_df['fee'] = pd.to_numeric(analysis_df['fee'], errors='coerce')
analysis_df['in_city'] = analysis_df['in_city'].apply(lambda x: 1 if x == '市内' else 0)
analysis_df = pd.merge(analysis_df , info[['visit_sn','gender','occupation_name','date_of_birth']], on='visit_sn', how='left')
analysis_df = analysis_df.rename(columns={'B15':'discharge_date'})
analysis_df['other_surgery_class4'] = analysis_df['other_surgery_class4'].apply(lambda x: 1 if x == '是' else 0)
analysis_df['is_surgery'] = analysis_df['is_surgery'].apply(lambda x: 1 if x == '是' else 0)
analysis_df['surgery_complication'] = analysis_df['surgery_complication'].apply(lambda x: 1 if x == '是' else 0)
analysis_df = analysis_df.rename(columns={'B12':'admission_date'})
analysis_df['RT_under_96h'] = (analysis_df['main_surgery_name'].fillna('').str.contains('呼吸机治疗[小于96',na=False,regex=False)| 
                               analysis_df['other_surgery_name'].fillna('').str.contains('呼吸机治疗[小于96',na=False,regex=False)).astype(int)
analysis_df['RT_over_96h'] = (analysis_df['main_surgery_name'].fillna('').str.contains('呼吸机治疗[大于等于96',na=False,regex=False)| 
                               analysis_df['other_surgery_name'].fillna('').str.contains('呼吸机治疗[大于等于96',na=False,regex=False)).astype(int)
analysis_df = pd.merge(analysis_df,facesheet[['visit_sn','C01C','C03C']],on='visit_sn',how='left',suffixes=('','_drop'))
analysis_df = analysis_df.loc[:,~analysis_df.columns.str.endswith("_drop")]
analysis_df = analysis_df.rename(columns={'C01C':'admission_diag_code','C03C':'discharge_diag_code'})
analysis_df['diag_combine'] = analysis_df['admission_diag_code'].str[0:3] + '-' + analysis_df['discharge_diag_code'].str[0:3]

def code_classification(input_code,cancer):
    result = 'other'
    for code in cancer:
        if (input_code[0:3]== code) | (input_code[4:7] == code):
            result = code
    return result
analysis_df['cancer_code'] = analysis_df['diag_combine'].apply(lambda x: code_classification(x,cancer_code))

analysis_df['ECMO'] = (analysis_df['main_surgery_name'].fillna('').str.contains('ECMO',na=False,regex=False)| 
                               analysis_df['other_surgery_name'].fillna('').str.contains('ECMO',na=False,regex=False)).astype(int)
analysis_df['CRRT'] = (analysis_df['main_surgery_name'].fillna('').str.contains('CRRT',na=False,regex=False)| 
                               analysis_df['other_surgery_name'].fillna('').str.contains('CRRT',na=False,regex=False)).astype(int)


In [ ]:

analysis_df[analysis_df['readmit_30d_emer_unplan'] == 1].iloc[5,:]

# analysis_df.to_feather(CACHEDIR + '/analysis_df.feather')

In [ ]:
note.info()

### Build a function filtering expressions in medical note

Use the function to distinguish ascites, fever, and positive bacterial microbiological cluture

In [ ]:
def note_filter(string, expression, neg_core, window):
    """
    Filter medical notes based on the presence of specific expressions within a defined timeframe around a reference event.
    
    Args:
        string: medical notes
        expression (list): List of expressions to search for in the notes.
        neg_core (list): List of negation terms to exclude notes containing them.
        window (int): Number of charecters before and after the expression to consider.
        
    Returns:
        true or false
    """

    import re
    neg_core = [neg.lower() for neg in neg_core]
    expression = [exp.lower() for exp in expression]
    string = string.lower()
    sentences = re.split(r'[，,；;。]', string)  # Split by Chinese punctuation
    for sentence in sentences:
     # Check each expression
        for exp in expression:
            if exp in sentence: 
                has_neg = False
                for neg in neg_core:
                    pattern = r'(?i)' + re.escape(neg) + r'(.{0,' + str(window) + r'})(' + re.escape(exp) + r')|(' + re.escape(exp) + r')(.{0,' + str(window) + r'})(.{0,' + str(window) + '})(' + re.escape(neg) + r')'
                    if re.search(pattern, sentence):
                        has_neg = True
                        break
                if not has_neg:
                    return True
    return False

In [ ]:
note_filter('未腹水，导致黄疸、胸水、凝血功能障碍、低白蛋白血症、肝性脑病等，可能需长期输白蛋白，',['腹水'],['导致','可能','如','好转','未'],10)

In [ ]:
NEG_CORE = ['导致','可能','如','好转','未见','无','没有','常规','未查','排除','未诉']
NOTE_TYPE = ['查房记录','阶段小结','接班记录','抢救记录','上级医师查房记录','首次病程记录','首次查房记录','术后记录','危急值处理记录','疑难病例讨论记录']

In [ ]:
note['腹水'] = note[note['record_title'].isin(NOTE_TYPE)]['record_content'].fillna('').apply(lambda x: note_filter(x,['腹水'],NEG_CORE,10))


TEST_NUM = 50
TEST_EXP = '腹水'
temp = note[note[TEST_EXP]==True][['record_content','record_sn']].iloc[0:TEST_NUM]
for x in range(TEST_NUM):
     string = temp.iloc[x]['record_content']
     sn = temp.iloc[x]['record_sn']
     strings = re.split(r'[，,；;。]', string)
     for sentence in strings:
         if TEST_EXP in sentence:
             print('sn:',sn,':',sentence)


In [ ]:
note['发热'] = note[note['record_title'].isin(NOTE_TYPE)]['record_content'].fillna('').apply(lambda x: note_filter(x,['发热'],NEG_CORE,20))


TEST_NUM = 50
TEST_EXP = '发热'
temp = note[note[TEST_EXP]==True][['record_content','record_sn']].iloc[0:TEST_NUM]
for x in range(TEST_NUM):
     string = temp.iloc[x]['record_content']
     sn = temp.iloc[x]['record_sn']
     strings = re.split(r'[，,；;。]', string)
     for sentence in strings:
         if TEST_EXP in sentence:
             print('sn:',sn,':',sentence)

In [ ]:
note['菌阳性'] = note[note['record_title'] == '危急值处理记录']['record_content'].fillna('').apply(lambda x: note_filter(x,['菌'],NEG_CORE,20))


TEST_NUM = 50
TEST_EXP = '菌'
temp = note[note['菌阳性']==True][['record_content','record_sn']].iloc[0:TEST_NUM]
for x in range(TEST_NUM):
     string = temp.iloc[x]['record_content']
     sn = temp.iloc[x]['record_sn']
     strings = re.split(r'[，,；;。]', string)
     for sentence in strings:
         if TEST_EXP in sentence:
             print('sn:',sn,':',sentence)

In [ ]:
note = note.rename(columns={'腹水':'ascites','发热':'fever','菌阳性':'positive_bacteria'}) # type: ignore
note.to_feather(CACHEDIR + '/note.feather')

### Combine Ascites, fever, and positive microbiological culture to the analysis dataframe

In [ ]:
analysis_df[['ascites_72h','fever_72h','positive_bacteria_72h']] = 0
for idx, row in analysis_df.iterrows():
    visit_sn = row['visit_sn']
    discharge_date = row['discharge_date']
    if pd.isna(discharge_date):
        continue
    start_window = discharge_date - pd.Timedelta(hours=72)
    notes_subset = note[(note['visit_sn'] == visit_sn) &  
                        (note['record_datetime'] >= start_window)]
    analysis_df.at[idx, 'ascites_72h'] = int(notes_subset['ascites'].any())
    analysis_df.at[idx, 'fever_72h'] = int(notes_subset['fever'].any())
    analysis_df.at[idx, 'positive_bacteria_72h'] = int(notes_subset['positive_bacteria'].any())
    if idx % 10000 == 0: # type: ignore
        print(f'Processed {idx} records')

In [ ]:
analysis_df.to_feather(CACHEDIR + '/analysis_df20251008.feather')

## Sklearn model

### Data preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score, average_precision_score, brier_score_loss
from sklearn.impute import SimpleImputer

In [ ]:
analysis_df = pd.read_feather(CACHEDIR + '/analysis_df20251008.feather')

In [ ]:

analysis_df[analysis_df['readmit_30d_emer_unplan'] == 1]['readmit_30d_emer_unplan'].sum()

In [ ]:
y = analysis_df['readmit_30d_emer_unplan_trans']
x = analysis_df.drop(columns=["visit_sn","patient_id","medical_record_no", "admission_date","discharge_date","surgery_date", "readmit_30d","readmit_30d_emer_unplan","readmit_30d_emer_unplan_trans",'unplaned','inpatient_type','date_of_birth','main_surgery_name','other_surgery_name','diag_combine'])

num_cols = x.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_cols = x.select_dtypes(include=['object']).columns.tolist()

num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, cat_cols)
])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state=42)


### Logistic Analysis

In [ ]:
from sklearn.linear_model import LogisticRegression

logit_model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced", solver="liblinear", penalty="l2"))
])


logit_model.fit(x_train, y_train)
y_pred_proba = logit_model.predict_proba(x_test)[:,1]

print("AUROC:", roc_auc_score(y_test, y_pred_proba))
print("AUPRC:", average_precision_score(y_test, y_pred_proba))
print("Brier score:", brier_score_loss(y_test, y_pred_proba))

### XGBOOST

In [ ]:
from xgboost import XGBClassifier

xgb_model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", XGBClassifier(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        min_child_weight=10,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight = (y==0).sum() / (y==1).sum(),  # unbalanced adjustment
        eval_metric="aucpr",
        tree_method="hist",
        n_jobs=-1,
        random_state=42
    ))
])

xgb_model.fit(x_train, y_train)
y_pred_proba = xgb_model.predict_proba(x_test)[:,1]

print("AUROC:", roc_auc_score(y_test, y_pred_proba))
print("AUPRC:", average_precision_score(y_test, y_pred_proba))
print("Brier score:", brier_score_loss(y_test, y_pred_proba))

### XGBoost Temp scaling

In [ ]:
from sklearn.calibration import CalibratedClassifierCV
calibrated_xgb = CalibratedClassifierCV(xgb_model.named_steps["model"], cv=5, method="sigmoid")
calibrated_xgb.fit(xgb_model.named_steps["preprocess"].transform(x_train), y_train)

y_pred_proba_cal = calibrated_xgb.predict_proba(xgb_model.named_steps["preprocess"].transform(x_test))[:,1]
print("Calibrated Brier:", brier_score_loss(y_test, y_pred_proba_cal))

### topk metric

In [ ]:
import numpy as np

K = [0.01, 0.03, 0.05, 0.075, 0.1]

def topk_metrics(y_true, y_prob, k=K):
    n = len(y_true)
    order = np.argsort(y_prob)[::-1]
    results = {}
    for rate in k:
        top_n = int(rate * n)
        idx = order[:top_n]
        recall = y_true.iloc[idx].sum() / y_true.sum()
        ppv = y_true.iloc[idx].sum() / top_n
        results[rate] = {'Recall': recall, 'Precision': ppv}
    return results

In [ ]:
y_pred_proba_xgb = xgb_model.predict_proba(x_test)[:, 1]
results_xgb = topk_metrics(y_test, y_pred_proba_xgb, k=K)
print(f'xgb result:{results_xgb}')

y_pred_proba_logit = logit_model.predict_proba(x_test)[:, 1]
results_logit = topk_metrics(y_test, y_pred_proba_logit, k=K)
print(f'logit result:{results_logit}')

In [ ]:
import matplotlib.pyplot as plt

k_vals = [round(k*100) for k in results_xgb.keys()]
recalls_xgb = [v['Recall'] for v in results_xgb.values()]
precisions_xgb = [v['Precision'] for v in results_xgb.values()]
recalls_logit = [v['Recall'] for v in results_logit.values()]
precisions_logit = [v['Precision'] for v in results_logit.values()]

plt.figure(figsize=(6,4))
plt.plot(k_vals, recalls_xgb, marker='o',linestyle='-', label='Recall-XGB')
plt.plot(k_vals, precisions_xgb, marker='s', linestyle='-',label='Precision-XGB')
plt.plot(k_vals, recalls_logit, marker='s',linestyle=':', label='Recall-logit')
plt.plot(k_vals, precisions_logit, marker='s', linestyle=':',label='Precision-logit')
plt.xlabel("Top-k (%)")
plt.ylabel("Metric Value")
plt.title("Top-k Recall & Precision (XGBoost)")
plt.legend()
plt.grid(True)
plt.show()